## Optiver Realized Volatility Project 

### Benchmark Calculation using pure past data

In [741]:
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
import time
from keras import optimizers



import pandas as pd
import plotly.express as px
train = pd.read_csv('Data/train.csv')
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [742]:
train[train['stock_id']==0]

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747
...,...,...,...
3825,0,32751,0.002611
3826,0,32753,0.001190
3827,0,32758,0.004264
3828,0,32763,0.004352


In [743]:
book_example = pd.read_parquet('Data/book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('Data/trade_train.parquet/stock_id=0')
stock_id = '0'
#book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
##trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id

In [744]:
book_example["time_id"].unique().shape

(3830,)

In [745]:
trade_example.shape

(123443, 6)

In [746]:
book_example['wap'] = (book_example['bid_price1'] * book_example['ask_size1'] +
                                book_example['ask_price1'] * book_example['bid_size1']) / (
                                       book_example['bid_size1']+ book_example['ask_size1'])

In [747]:
book_example_1 = book_example[book_example["time_id"]==5]

In [748]:
fig = px.line(book_example_1, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

In [749]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [750]:
book_example.loc[:,'log_return'] = log_return(book_example['wap'])
book_example = book_example[~book_example['log_return'].isnull()]
book_example

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,wap,log_return
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0,1.001448,0.000014
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000000
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,-0.000005
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,0.000000
5,5,11,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28,0,0.998515,0.000434
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28,0,0.998516,0.000001
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28,0,0.998516,0.000000
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28,0,0.998517,0.000001


In [751]:
df_returns = book_example.groupby("time_id")["log_return"].mean()
df_returns = pd.DataFrame(df_returns.values,index = df_returns.index )
df_returns = df_returns.rename(columns = {0:'log_returns'})
df_returns = df_returns.reset_index()
df_returns

,time_id,log_returns
0,5,7.613599e-06
1,11,-1.709393e-05
2,16,-1.438995e-05
3,31,-9.340946e-07
4,62,1.471116e-05
...,...,...
3825,32751,-9.373652e-06
3826,32753,1.949831e-05
3827,32758,-1.443173e-07
3828,32763,5.656307e-06


In [752]:
fig = px.line(df_returns, x="time_id", y="log_returns", title='Log return of stock_id_0, time_ids')
fig.show()

In [753]:
book_example_1

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,wap
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100,0,1.001434
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0,1.001448
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443
...,...,...,...,...,...,...,...,...,...,...,...,...
297,5,585,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0,1.003731
298,5,586,1.003129,1.003749,1.002612,1.003801,100,3,2,3,0,1.003731
299,5,587,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0,1.003731
300,5,588,1.003129,1.003749,1.002612,1.003801,100,3,2,3,0,1.003731


In [754]:
pd.DataFrame(book_example.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()

,time_id,log_return
0,5,0.004499
1,11,0.003966
2,16,0.002451
3,31,0.003742
4,62,0.003210
...,...,...
3825,32751,0.002660
3826,32753,0.004234
3827,32758,0.003368
3828,32763,0.003280


In [755]:
from statsmodels.tsa.stattools import adfuller
df_returns.log_returns.dropna(inplace=True)
adfuller(df_returns.log_returns)

(-26.015802746317878,
 0.0,
 8,
 3821,
 {'1%': -3.4320625617721574,
  '5%': -2.862296715736547,
  '10%': -2.567172809512658},
 -69887.33446734874)

In [756]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

Realized volatility for stock_id 0 on time_id 5 is 0.4764276101737872


In [840]:
import os
from sklearn.metrics import r2_score
import glob
list_order_book_file_train = glob.glob('Data/book_train.parquet')
list_order_trade_file_train = glob.glob('Data/trade_train.parquet')
list_order_book_file_test = glob.glob('Data/trade_train.parquet')
list_order_test_file_test = glob.glob('Data/trade_train.parquet')

In [765]:
def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [766]:
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized

In [767]:
#df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                         #  prediction_column_name='pred')

In [768]:
"""
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')
"""

"\ntrain['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)\ntrain = train[['row_id','target']]\ndf_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')\n"

In [769]:
"""
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')
"""

"\nfrom sklearn.metrics import r2_score\ndef rmspe(y_true, y_pred):\n    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))\nR2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)\nRMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)\nprint(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')\n"

### Recurrent Neural Network Predictions:

In [770]:
book_example

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,wap,log_return
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0,1.001448,0.000014
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000000
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,-0.000005
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0,1.001443,0.000000
5,5,11,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0,1.001448,0.000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
917548,32767,568,0.998275,0.998754,0.997796,0.998946,90,90,48,28,0,0.998515,0.000434
917549,32767,569,0.998275,0.998754,0.997892,0.998946,91,90,200,28,0,0.998516,0.000001
917550,32767,571,0.998275,0.998754,0.997892,0.998946,91,90,100,28,0,0.998516,0.000000
917551,32767,572,0.998275,0.998754,0.997892,0.998946,92,90,100,28,0,0.998517,0.000001


In [771]:
trade_example

,time_id,seconds_in_bucket,price,size,order_count,stock_id
0,5,21,1.002301,326,12,0
1,5,46,1.002778,128,4,0
2,5,50,1.002818,55,1,0
3,5,57,1.003155,121,5,0
4,5,68,1.003646,4,1,0
...,...,...,...,...,...,...
123438,32767,471,0.998659,200,3,0
123439,32767,517,0.998515,90,1,0
123440,32767,523,0.998563,1,1,0
123441,32767,542,0.998803,90,4,0


In [772]:
# convert an array of values into a dataset matrix
def create_dataset(dataset):
    dataX, dataY = [], []
    for t in  book_example.groupby(['time_id']):
        dataX.append((t[1]["log_return"]).values)
        dataY.append(t[0])

    return np.array(dataX), np.array(dataY)

In [773]:
data_x,data_y = create_dataset(book_example)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



### Create matrix for book_data

In [774]:
time_steps=[]
for i in range(data_x.shape[0]):
    
    time_steps.append(data_x[i].shape[0])

In [775]:
len(time_steps)

3830

In [776]:
book_example["seconds_in_bucket"].unique().max()

599

In [ ]:
def create_book_matrix(df):
    """
    create the matrix from the original book and trade data
    ARGS:
        df(dataframe): the original data to be converted to matrix
    Returns:
        mat_form- the matrix form of the dataframe
    """
    # used groupby to extract the time_id and seconds_in_bucket with log_return values
    book_mat = df.groupby(['time_id', 'seconds_in_bucket'])['log_return'].max().unstack()
    
    return book_mat.values
    

time_id
11    200
Name: seconds_in_bucket, dtype: int64

In [816]:
book_mat = book_example.groupby(['time_id', 'seconds_in_bucket'])['log_return'].max().unstack()

In [817]:
book_matrix = book_mat.values

In [780]:
book_matrix.shape

(3830, 600)

In [781]:
book_matrix

array([[            nan,  1.40690887e-05,             nan, ...,
                    nan,             nan,             nan],
       [-3.77902434e-03,             nan,             nan, ...,
                    nan,             nan,             nan],
       [-6.31105882e-04,  4.24686778e-04,  7.88683016e-05, ...,
                    nan,  1.16496605e-04,             nan],
       ...,
       [-1.69057994e-03,             nan, -2.01124061e-06, ...,
         8.50604112e-05,  0.00000000e+00,             nan],
       [ 1.21661279e-03,             nan,  0.00000000e+00, ...,
                    nan,             nan,             nan],
       [-2.05272090e-03,  1.73887198e-04, -1.61056717e-04, ...,
                    nan,             nan,             nan]])

### Trade Data combination with book_data

In [782]:
trade_example.loc[:,'log_return'] = log_return(trade_example['price'])
trade_example = trade_example[~trade_example['log_return'].isnull()]
trade_example

,time_id,seconds_in_bucket,price,size,order_count,stock_id,log_return
1,5,46,1.002778,128,4,0,0.000476
2,5,50,1.002818,55,1,0,0.000040
3,5,57,1.003155,121,5,0,0.000336
4,5,68,1.003646,4,1,0,0.000489
5,5,78,1.003762,134,5,0,0.000116
...,...,...,...,...,...,...,...
123438,32767,471,0.998659,200,3,0,0.000144
123439,32767,517,0.998515,90,1,0,-0.000144
123440,32767,523,0.998563,1,1,0,0.000048
123441,32767,542,0.998803,90,4,0,0.000240


In [ ]:
def create_trade_matrix(df):
    """
    create the matrix from the original book and trade data
    ARGS:
        df(dataframe): the original data to be converted to matrix
    Returns:
        mat_form- the matrix form of the dataframe
    """
    # used groupby to extract the time_id and seconds_in_bucket with log_return values
    book_mat = df.groupby(['time_id', 'seconds_in_bucket'])['log_return'].max().unstack().values
    
    return book_mat

In [783]:
trade_matrix =trade_example.groupby(['time_id', 'seconds_in_bucket'])['log_return'].max().unstack().values
trade_matrix.shape

(3830, 600)

In [ ]:
def scale_data(df_matrix):
    """
    scale data for using in deep learning algorithms(LSTM) using MinMaxScaler
    
    ARGS:
        df_matrix- numpy ndarray 
    Returns:
        scaled_data- numpy array of scaled data
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    scaled_data = scaler.fit_transform(df_matrix)
    
    return scaled_data    

In [785]:
scaler = MinMaxScaler(feature_range=(0, 1))
trade_scaled = scaler.fit_transform(trade_matrix)
book_scaled = scaler.fit_transform(book_matrix )
target_data = np.array(train[train["stock_id"]==0]["target"]).reshape(-1,1)
target_data = scaler.fit_transform(target_data)

In [786]:
#combine trade and book data log_returns 
multivariate_matrix =  np.array([book_scaled.T,trade_scaled.T]).T
multivariate_matrix.shape

(3830, 600, 2)

In [842]:
trial_matrix = multivariate_matrix[:,:,:]
trial_matrix.shape[2]

2

In [794]:
# define train and target data
trainX = trial_matrix
trainX = np.nan_to_num(trainX,nan=-1)
trainY = target_data[:]

In [795]:
n_steps = trainX.shape[1]
n_features = 2

In [819]:
trainX.shape[1]

600

### LSTM model:

In [ ]:
def LSTM_Model(n_steps,n_features):
    """
    implemet and fit the basic lstm model to predict the realized volatilty
    
    ARGS:
        
    Returns:
        model-the lstm model returned
    """
    model = Sequential()
    model.add(Masking(mask_value= -1, input_shape=(n_steps, n_features))) 
    model.add(LSTM(50,activation='tanh'))
    model.add(Dense(1))
    optimizer = optimizers.Adam(clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='mse')
    
    
    return model
    

In [ ]:
# data
list_order_book_file_train = glob.glob('Data/book_train.parquet')
list_order_trade_file_train = glob.glob('Data/trade_train.parquet')
list_order_book_file_test = glob.glob('Data/trade_train.parquet')
list_order_trade_file_test = glob.glob('Data/trade_train.parquet')

In [ ]:
def realized_volatility_per_time_id(book_file_path,trade_file_path,book_test_file_path,trade_test_file_path, prediction_column_name):
    
    # create matrix for book_data log returns based on time_id and seconds in bucket as rows and columns
    df_book_data = pd.read_parquet(book_file_path)
    
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    
    book_matrix = create_book_matrix(df_book_data)
    
    # create trade matrix of log_returns 
    df_trade_data = pd.read_parquet(trade_file_path)
    
    df_trade_data.loc[:,'log_return'] = log_return(df_trade_data['price'])
    df_trade_data = df_trade_data[~df_trade_data['log_return'].isnull()]
    
    trade_matrix = create_trade_matrix(df_trade_data)
    
    # scale the data to prepare for machine learning
    book_scaled = scale_data(book_matrix)
    trade_scaled = scale_data(trade_matrix)
    
    # create target data from train data and scale it 
    target_data = np.array(train["target"]).reshape(-1,1)
    target_data = scale_data(target_data)
    
    #combine trade and book data log_returns as one numpy array(tensor) with three dimensions 
    book_trade_matrix =  np.array([book_scaled.T,trade_scaled.T]).T # input data for LSTM

    # define train and target and convert nans in trainX to numerical -1 
    trainX = book_trade_matrix
    trainX = np.nan_to_num(trainX,nan=-1)
    trainY = target_data[:]
    
    # determine the LSTM parameters to learning the model
    n_steps = trainX.shape[1]
    n_features = book_trade_matrix.shape[2]
    
    # initialize the LSTM model
    model = LSTM_Model(n_steps,n_features)
    
    
    # fit the model with train and target
    model.fit(trainX, trainY, epochs=25, verbose=0)
    
    
    
    #prepare the test data as above  
    df_book_test = pd.read_parquet(book_test_file_path)
    
    # create matrix for book_test_data log returns based on time_id and seconds in bucket as rows and columns    
    df_book_test['wap'] =(df_book_test['bid_price1'] * df_book_test['ask_size1']+df_book_test['ask_price1'] * df_book_test['bid_size1'])  / (
                                      df_book_test['bid_size1']+ df_book_test[
                                  'ask_size1'])
    df_book_test['log_return'] = df_book_test.groupby(['time_id'])['wap'].apply(log_return)
    df_book_test = df_book_test[~df_book_data['log_return'].isnull()]
    
    test_book_matrix = create_book_matrix(df_book_test)
    
    # create trade matrix of log_returns 
    df_trade_test = pd.read_parquet(trade_test_file_path)
    df_trade_test.loc[:,'log_return'] = log_return(df_trade_test['price'])
    df_trade_test = df_trade_test[~df_trade_test['log_return'].isnull()]
    
    test_trade_matrix = create_trade_matrix(df_trade_test)
    
    # scale the data to prepare for machine learning
    book_test_scaled = scale_data(test_book_matrix)
    trade_test_scaled = scale_data(test_trade_matrix)
    
    # create target data from train data and scale it 
    #target_test = np.array(test["target"]).reshape(-1,1)
    #target_test = scale_data(target_test)
    
    #combine trade and book data log_returns as one numpy array(tensor) with three dimensions 
    book_trade_matrix =  np.array([book_scaled.T,trade_scaled.T]).T # input data for LSTM
    
     # define train and target data
    testX = trial_matrix
    testX = np.nan_to_num(testX,nan=-1)
    #testY = target_test[:]
    
    
    # predict model results for test data
    test_preds = model.predict(testX, verbose=0)
    
    preds_transformed = scaler.inverse_transform(test_preds)
    #test_y_transformed = scaler.inverse_transform(trainY)
    
    preds_transformed = preds_transformed.reshape((preds_transformed.shape[0],))
    

    
    df_realized_vol_per_stock =  pd.DataFrame(preds_transformed)
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {0:prediction_column_name})
    stock_id = book_test_file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]




In [ ]:
def realized_volatility_per_stock(book_train_list,trade_train_list,book_test_list,trade_test_list,
                                  prediction_column_name):
    df_past_realized = pd.DataFrame()
    
    for book in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized

In [ ]:
df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                         #  prediction_column_name='pred')

In [832]:
# define model
model = Sequential()
model.add(Masking(mask_value= -1, input_shape=(n_steps, n_features))) 
model.add(LSTM(50,activation='tanh'))
model.add(Dense(1))
optimizer = optimizers.Adam(clipvalue=0.5)
model.compile(optimizer=optimizer, loss='mse')

In [833]:
%%timeit
model.fit(trainX, trainY, epochs=25, verbose=1)

Epoch 1/25
3830/3830 [==============================] - 53s 14ms/step - loss: 0.0098
Epoch 2/25
3830/3830 [==============================] - 48s 12ms/step - loss: 0.0065
Epoch 3/25
3830/3830 [==============================] - 46s 12ms/step - loss: 0.0065
Epoch 4/25
3830/3830 [==============================] - 47s 12ms/step - loss: 0.0064
Epoch 5/25
3830/3830 [==============================] - 47s 12ms/step - loss: 0.0064
Epoch 6/25
3830/3830 [==============================] - 47s 12ms/step - loss: 0.0065
Epoch 7/25
3830/3830 [==============================] - 50s 13ms/step - loss: 0.0064
Epoch 8/25
3830/3830 [==============================] - 53s 14ms/step - loss: 0.0065
Epoch 9/25
3830/3830 [==============================] - 48s 13ms/step - loss: 0.0065
Epoch 10/25
3830/3830 [==============================] - 47s 12ms/step - loss: 0.0064
Epoch 11/25
3830/3830 [==============================] - 48s 13ms/step - loss: 0.0064
Epoch 12/25
3830/3830 [==============================] - 47s 12

3830/3830 [==============================] - 47s 12ms/step - loss: 0.0034
Epoch 22/25
3830/3830 [==============================] - 49s 13ms/step - loss: 0.0043
Epoch 23/25
3830/3830 [==============================] - 47s 12ms/step - loss: 0.0035
Epoch 24/25
3830/3830 [==============================] - 47s 12ms/step - loss: 0.0033
Epoch 25/25
3830/3830 [==============================] - 47s 12ms/step - loss: 0.0032
Epoch 1/25
3830/3830 [==============================] - 49s 13ms/step - loss: 0.0037
Epoch 2/25
3830/3830 [==============================] - 46s 12ms/step - loss: 0.0032
Epoch 3/25
3830/3830 [==============================] - 48s 12ms/step - loss: 0.0029
Epoch 4/25
3830/3830 [==============================] - 50s 13ms/step - loss: 0.0032
Epoch 5/25
3830/3830 [==============================] - 47s 12ms/step - loss: 0.0029
Epoch 6/25
3830/3830 [==============================] - 48s 12ms/step - loss: 0.0028
Epoch 7/25
3830/3830 [==============================] - 46s 12ms/step - 

3830/3830 [==============================] - 46s 12ms/step - loss: 0.0020
Epoch 17/25
3830/3830 [==============================] - 44s 12ms/step - loss: 0.0019
Epoch 18/25
3830/3830 [==============================] - 45s 12ms/step - loss: 0.0020
Epoch 19/25
3830/3830 [==============================] - 45s 12ms/step - loss: 0.0020
Epoch 20/25
3830/3830 [==============================] - 44s 12ms/step - loss: 0.0020
Epoch 21/25
3830/3830 [==============================] - 45s 12ms/step - loss: 0.0019
Epoch 22/25
3830/3830 [==============================] - 45s 12ms/step - loss: 0.0020
Epoch 23/25
3830/3830 [==============================] - 45s 12ms/step - loss: 0.0024
Epoch 24/25
3830/3830 [==============================] - 45s 12ms/step - loss: 0.0023
Epoch 25/25
3830/3830 [==============================] - 45s 12ms/step - loss: 0.0020
19min 18s ± 31.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [834]:
train_preds = model.predict(trainX, verbose=0)

In [835]:
preds_transformed = scaler.inverse_transform(train_preds)
train_y_transformed = scaler.inverse_transform(trainY)

In [845]:
preds_transformed.shape[0]

(3830, 1)

In [844]:
preds_transformed.reshape(())

array([[0.00329138],
       [0.00266221],
       [0.00269293],
       ...,
       [0.00257146],
       [0.00305992],
       [0.00255922]], dtype=float32)

In [846]:
preds_transformed.reshape((preds_transformed.shape[0],))

array([0.00329138, 0.00266221, 0.00269293, ..., 0.00257146, 0.00305992,
       0.00255922], dtype=float32)

In [836]:
RMSPE = round(rmspe(y_true = train_y_transformed, y_pred = preds_transformed),3)
print(f'Performance of the naive prediction:  RMSPE: {RMSPE}')

Performance of the naive prediction:  RMSPE: 0.433


In [838]:
R2 = round(r2_score(y_true = train_y_transformed, y_pred = preds_transformed),3)
R2

0.692

In [ ]:
"""
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
dataframe = read_csv('airline-passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
"""